### This notebook is produced for Coventry University's 7135CEM Coursework
Made by Sunggu Choi

The dataset is accessiable on https://www.kaggle.com/thoughtvector/customer-support-on-twitter
    
**Part 1 - Exploratory Analysis and Topic Modelling**

Identitfy the topic is positive or negitive, also predict how the answer would be.

In [1]:
#!pip3 install wordcloud
#!pip3 install gensim
# !pip3 install pyLDAvis
# !pip3 install numexpr

In [2]:
# Import requires libraries
import nltk
import pandas as pd

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from tqdm import tqdm_notebook
import seaborn
from datetime import datetime
from matplotlib import pyplot as plt

import gensim
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
from gensim.models import LdaModel
from pprint import pprint


%matplotlib inline
%config InlineBackend.figure_format = 'svg'

C:\Users\chois11\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


# Change dataset type for Gensim

In [3]:
df_dataset = pd.read_csv("./processed.csv")

In [4]:
df_dataset["text"] = df_dataset["text"].map(lambda x: x.split(' '))

In [5]:
df_dataset

,text
0,"[['understand',, 'would',, 'like',, 'assist',,..."
1,"[['sprintcar',, 'propos']]"
2,"[['sprintcar',, 'sent',, 'sever',, 'privat',, ..."
3,"[['pleas',, 'send',, 'us',, 'privat',, 'messag..."
4,[['sprintcar']]
...,...
2811769,"[['hey',, ""'d"",, 'happi',, 'look',, 'pleas',, ..."
2811770,"[['wtf',, '’',, 'realli',, 'shitti',, 'servic'..."
2811771,"[['sprintcar',, 'go',, 'co',, 'vtmhetlq',, 'as..."
2811772,"[['sound',, 'delici',, 'sarah',, '😋',, 'co',, ..."


In [6]:
# Create Dictionary
id2word = corpora.Dictionary(df_dataset['text'])

# Create Corpus
texts = df_dataset['text']

# Corpus
corpus = [id2word.doc2bow(text) for text in texts]

In [7]:
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1)]


In [9]:
from gensim.models import LdaModel
from pprint import pprint

corpus = corpus
id2word = id2word

# Set training parameters
num_topics = 5
chunksize = 100000
passes = 2 # Since the datset is huge
iterations = 400
eval_every = 1


import gensim
import logging # This allows for seeing if the model converges. A log file is created.
logging.basicConfig(filename='./lda_model.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    # Build LDA model
lda_model = LdaModel(corpus=corpus,
                   id2word=id2word,
                   num_topics=num_topics,
                   chunksize=chunksize,
                   alpha = 'auto',
                   eta = 'auto',
                   random_state=100000,
                   passes=passes,
                   eval_every = eval_every)

lda_model.save('lda_model.model') # Save LDA model



In [10]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.029*"\'co\'," + 0.021*"\'’\'," + 0.009*"\'get\'," + 0.008*""\'s"," + '
  '0.008*"\'store\'," + 0.008*""n\'t"," + 0.007*"\'thank\'," + '
  '0.007*"\'like\'," + 0.006*"\'one\'," + 0.005*"\'love\',"'),
 (1,
  '0.012*"\'’\'," + 0.011*""n\'t"," + 0.011*"\'servic\'," + '
  '0.010*"\'flight\'," + 0.010*"[\'amazonhelp\'," + 0.010*"\'custom\'," + '
  '0.009*"\'order\'," + 0.009*"\'time\'," + 0.008*"\'get\'," + '
  '0.007*"\'day\',"'),
 (2,
  '0.026*"\'de\'," + 0.018*"[\'amazonhelp\'," + 0.017*"\'co\'," + '
  '0.016*"\'que\'," + 0.012*"\'la\'," + 0.011*"\'en\'," + 0.009*"\'el\'," + '
  '0.009*"\'un\'," + 0.007*"\'le\'," + 0.007*"\'por\',"'),
 (3,
  '0.024*"\'’\'," + 0.013*"\'support\'," + 0.012*"\'co\'," + 0.011*"\'get\'," '
  '+ 0.010*"\'tri\'," + 0.010*""n\'t"," + 0.010*"\'app\'," + '
  '0.010*"[\'applesupport\'," + 0.009*"\'work\'," + 0.009*"\'updat\',"'),
 (4,
  '0.043*"\'us\'," + 0.038*"\'co\'," + 0.032*"\'dm\'," + 0.031*"\'pleas\'," + '
  '0.016*"\'help\'," + 0.016*"[\'hi\'," + 0

In [11]:
# Load saved model

from gensim.test.utils import datapath

# Load a potentially pretrained model from disk.
lda_Model = LdaModel.load('lda_model.model')

In [12]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# feed the LDA model into the pyLDAvis instance
lda_viz = gensimvis.prepare(lda_Model, corpus, id2word) # Load model and visualise

pyLDAvis.enable_notebook()


In [13]:
pyLDAvis.display(lda_viz)

C:\Users\chois11\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
pyLDAvis.save_html()

C:\Users\chois11\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


TypeError: save_html() missing 2 required positional arguments: 'data' and 'fileobj'

In [ ]:
print("perplexity:" , lda_Model.log_perplexity(corpus))

C:\Users\chois11\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
coherence_model_lda = models.CoherenceModel(model = lda_Model, texts
                                           -*)